In this notebook I will explore and augment a custom dataset for entity recognition of cryptocurrencies scraped from reddit using my other project Dragonfly.

https://huggingface.co/datasets/conll2003
https://www.freecodecamp.org/news/getting-started-with-ner-models-using-huggingface/

https://colab.research.google.com/github/abhimishra91/transformers-tutorials/blob/master/transformers_ner.ipynb#scrollTo=IV72GFgq_ZYb

https://github.com/nlpyang/BertSum

https://github.com/abhimishra91/transformers-tutorials

In [14]:
%load_ext autoreload
%autoreload 2
import os
import tqdm
from typing import List
from transformers import AutoTokenizer
from dotenv import load_dotenv
load_dotenv()
from pymongo import MongoClient
client = MongoClient(os.getenv("MONGO_CONNECTION_STRING"))
db = client["dragonfly"]

import torch
from torch.utils.data import Dataset
from train.dragonfly_dataset.src import DragonflyDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# hyperparameters
MODEL_NAME = "distilbert-base-cased"
SEQ_MAX_LENGTH = 512
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [20]:
dt = DragonflyDataset(tokenizer, SEQ_MAX_LENGTH)

INFO:DragonflyDataset:Downloading raw data from Dragonfly collection.
10it [00:00, 13.69it/s]
INFO:DragonflyDataset:Tokenizing text.
INFO:DragonflyDataset:Bulding NER tags.
1it [00:00, 1985.00it/s]


In [23]:
len(dt.dataset["inputs"]["input_ids"])

7

In [21]:
for tok, tag in zip(tokenizer.convert_ids_to_tokens(dt.dataset["inputs"]["input_ids"][1]), dt.dataset["tags"][1]):
    print(tok,tag)

IndexError: list index out of range

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'CRYPTO',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',

In [80]:
del dataset

In [51]:
tokens = tokenizer(
    ["Hello this [CLS] is sparta."],
    padding="max_length",
    truncation=True,
    return_tensors="pt",
    max_length = 10,
    return_offsets_mapping=True
    )
print(tokens['input_ids'])
tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])

tensor([[  101,  8667,  1142,   101,  1110, 22620, 16242,   119,   102,     0]])


['[CLS]',
 'Hello',
 'this',
 '[CLS]',
 'is',
 'spa',
 '##rta',
 '.',
 '[SEP]',
 '[PAD]']

In [ ]:
class CryptoDataset(Dataset):
    def __init__(self):
        pass

    def __getitem__(self,index):
        pass

    def __len__(self):
        pass

In [36]:
# 1. Extraction of the latest scraped (reddit) data from the databse


In [73]:
# O, I-CRYPTO, CRYPTO
dataset["tags"] = []
for token_ids, offsets, matches in tqdm.tqdm(zip(dataset["inputs"]["input_ids"],dataset["inputs"]["offset_mapping"], dataset["ner_matches"])):
    tags = []
    for token,offset in zip(tokenizer.convert_ids_to_tokens(token_ids),offsets):
        if token in ["[CLS]","[SEP]","[PAD]"]:
            tags.append("O")
            continue
        tag = "O"
        for match in matches:
            if tag == "O":
                # print(token,offset,match["span"][0],offset[0] == match["span"][0],offset[0] > match["span"][0] and offset[1] <= match["span"][1])
                if offset[0] == match["span"][0]:
                    tag = "I-CRYPTO"
                elif offset[0] > match["span"][0] and offset[1] <= match["span"][1]:
                    tag = "CRYPTO"
                else:
                    tag = "O"
        tags.append(tag)

13560it [00:31, 436.51it/s]


In [78]:
tag2id = {
    "O":0,
    "I-CRYPTO":1,
    "CRYPTO":2
}
dataset["encoded_tags"] = [tag2id(x) for x in dataset["tags"]]

In [79]:
dataset.keys()

dict_keys(['text', 'ner_matches', 'inputs', 'tags', 'encoded_tags'])

In [43]:
offsets[:40]

tensor([[  0,   0],
        [  0,   3],
        [  4,  11],
        [ 12,  15],
        [ 16,  20],
        [ 21,  23],
        [ 24,  28],
        [ 29,  32],
        [ 33,  35],
        [ 36,  38],
        [ 39,  42],
        [ 43,  47],
        [ 48,  52],
        [ 53,  56],
        [ 57,  65],
        [ 66,  68],
        [ 69,  71],
        [ 71,  75],
        [ 76,  79],
        [ 79,  80],
        [ 80,  81],
        [ 82,  86],
        [ 87,  89],
        [ 89,  91],
        [ 92,  94],
        [ 96,  98],
        [ 98, 102],
        [103, 105],
        [106, 113],
        [113, 114],
        [114, 116],
        [116, 120],
        [120, 121],
        [122, 128],
        [129, 131],
        [132, 133],
        [133, 135],
        [135, 136],
        [  0,   0],
        [  0,   0]])

In [66]:
for token, tag in zip(tokenizer.convert_ids_to_tokens(input_ids), tags):
    print(token,tag)

[CLS] O
The O
biggest O
red O
flag O
as O
such O
for O
me O
is O
the O
fact O
that O
the O
projects O
on O
Co O
##smos O
don O
’ O
t O
need O
AT I-CRYPTO
##OM CRYPTO
or O
Co O
##smos O
to O
survive O
/ O
ex O
##sist O
, O
unlike O
on O
D I-CRYPTO
##OT CRYPTO
. O
[SEP] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[PAD] O
[

In [45]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def tokenize(raw_texts : List[str], tokenizer):
    return tokenizer(raw_texts, padding="max_length", truncation=True)
tokenized_texts = tokenize([d["raw_text"] for d in dataset],tokenizer)

Downloading: 100%|███████████████████████████| 28.0/28.0 [00:00<00:00, 18.1kB/s]
Downloading: 100%|██████████████████████████████| 483/483 [00:00<00:00, 196kB/s]
Downloading: 100%|████████████████████████████| 226k/226k [00:00<00:00, 763kB/s]
Downloading: 100%|████████████████████████████| 455k/455k [00:00<00:00, 798kB/s]


In [43]:
for i, data in enumerate(dataset):
    ner_tags = []
    character_count = 0
    token_i = 0
    for token in tokenized_texts[i].tokens:
        new_tag = 0
        if token not in ["[PAD]","[CLS]","[SEP]"]:
            for tag in data["ner_matches"]:
                if token == tag["match"].lower():
                    new_tag = 1
        ner_tags.append(new_tag)
    data["ner_tags"] = ner_tags 

In [9]:
data

{'id': 0,
 'raw_text': 'The biggest red flag as such for me is the fact that the projects on Cosmos don’t need ATOM or  Cosmos to survive/exsist, unlike on DOT.',
 'ner_tags': [{'symbol': 'DOT', 'match': 'DOT', 'span': [132, 135]},
  {'symbol': 'ATOM', 'match': 'ATOM', 'span': [87, 91]}]}

36

In [ ]:
# 2. BERT pre-processing: tookenization and building labels